#IMPORTING LIBRARIES AND MOUNTING DRIVE



In [3]:
%pip install biopython
%pip install silence_tensorflow

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error
from Bio.PDB import *

path='./'

import sys
sys.path.append("/home/liang/code/acdc-nn/acdc_nn")
import util
import nn

#REPRODUCING PAPER RESULTS

##ACDC-NN

In the following cell we run a loop on the 8 cross-validation folds generated with blustclust. The network has been trained in transfer learning on s2648 and Ivankov, therefore the cv sets have been generated taking into account the similarity between proteins (Similarity < 25%).

Then we load the weights of the network for each fold, generate protein structures, create the input for the network in the appropriate form and also generate the reverse mutation. Finally we predict the $\Delta \Delta G$ with ACDC-NN. 
We have done the same thing for both direct and inverse proteins.
We underlie that in the following cell we are using ACDC-NN with one structure.


In [3]:
#path: ./replicate_results/

cv_folds=[0,1,2,4,5,6,7,8] # cross-validation folds

cv_pred_dir=list()
cv_pred_inv=list()

for fold in cv_folds:

    pred_dir=list()
    pred_inv=list()

    #loading the proper fold
    ssym_dir=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_dir_{}.mut'.format(fold), sep=' ',header=None)
    ssym_inv=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_inv_{}.mut'.format(fold), sep=' ',header=None)
    
    # building the specific model with cv weights
    path_weights=path+"weights_cv/Weights_PostTL_CV_{}".format(fold)
    num_H=[32,16]
    d=0.2
    ACDC_NN=nn.build_acdc_3d(num_H,d,25)[0]
    ACDC_NN.load_weights(path_weights)

    #Ssym dir prediction
    for protein,mut in zip(list(ssym_dir[0]),list(ssym_dir[1])):

      prof_path =path + 'profiles/' + protein +'.prof.gz' 
      pdb_path= path + 'pdbs/' + protein[:-1] +'.pdb.gz'
      chain = protein[-1]

      # information processing
      # get structure and other information
      structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
      prof = util.getProfile(prof_path)

      kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
      kvar_pdb=(mut[0],mut[1:-1],mut[-1])

      dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
      list_dist_neigh_3d = dist_neigh_3d[kvar]

      # extracting features
      codif=util.getMutCod(mut)
      all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
     
      #dir 
      To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T

      #inv (dir)
      To_predict_inv=To_predict_dir.copy()
      To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
      
      # Making input in the proper shape 
      Xm_d, X1D_d, X3D_d = nn.mkInp_3d(np.asarray(To_predict_dir).astype(np.float32),500)
      Xm_i, X1D_i, X3D_i = nn.mkInp_3d(np.asarray(To_predict_inv).astype(np.float32),500)  
  
      #predict
      prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
      pred_dir.append(prediction[0][0][0])

    #Ssym inv prediction
    for protein,mut in zip(list(ssym_inv[0]),list(ssym_inv[1])):
      prof_path =path + 'profiles/' + protein +'.prof.gz' 
      pdb_path= path + 'pdbs/'  + protein[:-1] +'.pdb.gz'
      chain = protein[-1]

      # information processing
      # get structure and other information
      structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
      prof = util.getProfile(prof_path)

      kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
      kvar_pdb=(mut[0],mut[1:-1],mut[-1])

      dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
      list_dist_neigh_3d = dist_neigh_3d[kvar]

      # extracting features
      codif=util.getMutCod(mut)
      all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
     
      #dir
      To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T
      
      #dir (inv)
      To_predict_inv=To_predict_dir.copy()
      To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
      
      # Making input in the proper shape 
      Xm_d, X1D_d, X3D_d = nn.mkInp_3dp(np.asarray(To_predict_dir).astype(np.float32),500)
      Xm_i, X1D_i, X3D_i = nn.mkInp_3d(np.asarray(To_predict_inv).astype(np.float32),500)  
  
      #predict
      prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
      pred_inv.append(prediction[0][0][0])
    
    #appending results
    cv_pred_dir.append(pred_dir)
    cv_pred_inv.append(pred_inv)

#merging the results

cv_pred_dir=[protein for cv in cv_pred_dir for protein in cv] 
cv_pred_inv=[protein for cv in cv_pred_inv for protein in cv]

Traceback (most recent call last):
  File "/home/liang/anaconda3/envs/ddgun/lib/python3.7/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 519, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined
Traceback (most recent call last):
  File "/home/liang/anaconda3/envs/ddgun/lib/python3.7/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 519, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined


AttributeError: module 'nn' has no attribute 'mkInp_3dp'

RESHAPING RESULTS

In the following cell we merge together all the cv folds and the results obtained, building a dataframe so that we can easily measure performances

In [8]:
# fold 0

S_dir=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_dir_0.mut', sep=' ',header=None)
S_inv=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_inv_0.mut', sep=' ',header=None)

# appending the others

cv_folds=[1,2,4,5,6,7,8] # cross-validation folds

for fold in cv_folds:
    S_dir=S_dir.append(pd.read_csv(path+'Ssym_cv.mut/'+'ssym_TS_dir_{}.mut'.format(fold), sep=' ',header=None),)
    S_inv=S_inv.append(pd.read_csv(path+'Ssym_cv.mut/'+'ssym_TS_inv_{}.mut'.format(fold), sep=' ',header=None),)

S_dir.columns=['Protein','Mut','DDG']
S_inv.columns=['Protein','Mut','DDG']

S_dir['DDG_pred']=cv_pred_dir
S_inv['DDG_pred']=cv_pred_inv


MEASURE OF ACDC-NN PERFORMANCE ON SSYM 

Ssym direct 

In [9]:
print('pearson_direct : ', np.corrcoef(S_dir['DDG_pred'],S_dir['DDG'])[0][1].round(2))
print('rmse : ',round(math.sqrt(mean_squared_error(S_dir['DDG_pred'],S_dir['DDG'])),2))

pearson_direct :  0.58
rmse :  1.42


Ssym inverse 

In [10]:
print('pearson_inverse : ', np.corrcoef(S_inv['DDG_pred'],S_inv['DDG'])[0][1].round(2))
print('rmse : ',round(math.sqrt(mean_squared_error(S_inv['DDG_pred'],S_inv['DDG'])),2))

pearson_inverse :  0.55
rmse :  1.47


Antisimmetry

In [11]:
print('r_dir-inv : ' ,np.corrcoef(cv_pred_dir,cv_pred_inv)[0][1].round(2))
print('bias : ', util.bias(cv_pred_dir,cv_pred_inv).round(2))

r_dir-inv :  -0.99
bias :  -0.01


##ACDC-NN* (two structures available)

In [14]:
#path
cv_folds=[0,1,2,4,5,6,7,8] # cross-validation folds

cv_pred_dir=list()
cv_pred_inv=list()

for fold in cv_folds:

    pred_dir=list()
    pred_inv=list()

    #loading the proper fold
    ssym_dir=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_dir_{}.mut'.format(fold), sep=' ',header=None)
    ssym_inv=pd.read_csv(path+'Ssym_cv.mut/ssym_TS_inv_{}.mut'.format(fold), sep=' ',header=None)
    
    # building the specific model with cv weights
    path_weights=path+"weights_cv/Weights_PostTL_CV_{}".format(fold)
    num_H=[32,16]
    d=0.2
    ACDC_NN=nn.build_acdc_3d(num_H,d,25)[0]
    ACDC_NN.load_weights(path_weights)

    #Ssym dir-inv prediction using both structures
    for (protein_dir,protein_inv,mut_dir,mut_inv) in zip(list(ssym_dir[0]),list(ssym_inv[0]),list(ssym_dir[1]),list(ssym_inv[1])):

      # information processing
      # get structure and other information for the direct protein

      prof_path_dir =path + 'profiles/' + protein_dir +'.prof.gz' 
      pdb_path_dir= path + 'pdbs/' + protein_dir[:-1] +'.pdb.gz'
      chain_dir = protein_dir[-1]

      structure_dir, pchain_dir, seq_dir, d_seq2pdb_dir, d_pdb2seq_dir  = util.pdb2info(pdb_path_dir, chain_dir)
      prof_dir = util.getProfile(prof_path_dir)

      kvar_dir=(mut_dir[0],d_pdb2seq_dir[mut_dir[1:-1]],mut_dir[-1])
      kvar_pdb_dir=(mut_dir[0],mut_dir[1:-1],mut_dir[-1])

      dist_neigh_3d_dir= util.get_neigh_ps(kvar_pdb_dir,5,d_seq2pdb_dir,pchain_dir) 
      list_dist_neigh_3d_dir = dist_neigh_3d_dir[kvar_dir]

      # extracting features
      codif_dir=util.getMutCod(mut_dir)
      all_profile_dir = util.Unified_prof(kvar_dir[1],prof_dir,seq_dir, list_dist_neigh_3d_dir)
     
      #dir 
      To_predict_dir=pd.DataFrame([*codif_dir,*all_profile_dir,*np.zeros(600-len(all_profile_dir))]).T

      # information processing
      # get structure and other information for the inverse protein
      prof_path_inv =path + 'profiles/' + protein_inv +'.prof.gz' 
      pdb_path_inv= path + 'pdbs/' + protein_inv[:-1] +'.pdb.gz'
      chain_inv = protein_inv[-1]

      # information processing
      # get structure and other information for the inverse protein

      structure_inv, pchain_inv, seq_inv, d_seq2pdb_inv, d_pdb2seq_inv  = util.pdb2info(pdb_path_inv, chain_inv)
      prof_inv = util.getProfile(prof_path_inv)

      kvar_inv=(mut_inv[0],d_pdb2seq_inv[mut_inv[1:-1]],mut_inv[-1])
      kvar_pdb_inv=(mut_inv[0],mut_inv[1:-1],mut_inv[-1])

      dist_neigh_3d_inv= util.get_neigh_ps(kvar_pdb_inv,5,d_seq2pdb_inv,pchain_inv) 
      list_dist_neigh_3d_inv = dist_neigh_3d_inv[kvar_inv]

      # extracting features
      codif_inv=util.getMutCod(mut_inv)
      all_profile_inv = util.Unified_prof(kvar_inv[1],prof_inv,seq_inv, list_dist_neigh_3d_inv)
     
      #inv
      To_predict_inv=pd.DataFrame([*codif_inv,*all_profile_inv,*np.zeros(600-len(all_profile_inv))]).T
      

      # Making input in the proper shape 
      Xm_d, X1D_d, X3D_d = nn.mkInp_3d(np.asarray(To_predict_dir).astype(np.float32),500)
      Xm_i, X1D_i, X3D_i = nn.mkInp_3d(np.asarray(To_predict_inv).astype(np.float32),500)  
  
      #predict dir
      prediction_dir=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
      pred_dir.append(prediction_dir[0][0][0])

      #predict inv
      prediction_inv=ACDC_NN.predict([X3D_i, X1D_i, Xm_i , X3D_d, X1D_d, Xm_d])
      pred_inv.append(prediction_inv[0][0][0])    

    #appending results
    cv_pred_dir.append(pred_dir)
    cv_pred_inv.append(pred_inv)

#merging the results

cv_pred_dir=[protein for cv in cv_pred_dir for protein in cv]
cv_pred_inv=[protein for cv in cv_pred_inv for protein in cv]

1/1 [==============================] - 0s 30ms/step


/home/liang/anaconda3/envs/ddgun/lib/python3.7/site-packages/Bio/PDB/Polypeptide.py:344: UserWarning: Assuming residue  CA is an unknown modified amino acid
  % residue.get_resname()


1/1 [==============================] - 0s 20ms/step


/home/liang/anaconda3/envs/ddgun/lib/python3.7/site-packages/Bio/PDB/Polypeptide.py:344: UserWarning: Assuming residue  CA is an unknown modified amino acid
  % residue.get_resname()


1/1 [==============================] - 0s 21ms/step


ADDING ACDC-NN* PREDICTIONS TO THE PREVIOUS DATAFRAMES

In [15]:
S_dir['DDG_pred_two_pdbs']=cv_pred_dir
S_inv['DDG_pred_two_pdbs']=cv_pred_inv


MEASURE OF ACDC-NN* PERFORMANCE ON SSYM 

Ssym direct 

In [16]:
print('pearson dir: ',np.corrcoef(S_dir['DDG'],S_dir['DDG_pred_two_pdbs'])[0][1].round(2))
print('rmse dir: ',round(math.sqrt(mean_squared_error(S_dir['DDG'],S_dir['DDG_pred_two_pdbs'])),2))

pearson dir:  0.57
rmse dir:  1.45


Ssym inverse

In [17]:
print('pearson inv: ',np.corrcoef(S_inv['DDG'],S_inv['DDG_pred_two_pdbs'])[0][1].round(2))
print('rmse inv: ',round(math.sqrt(mean_squared_error(S_inv['DDG'],S_inv['DDG_pred_two_pdbs'])),2))

pearson inv:  0.57
rmse inv:  1.45


Antisimmetry

In [19]:
print('r_dir-inv: ' ,np.corrcoef(cv_pred_dir,cv_pred_inv)[0][1].round(2))
print('bias: ', util.bias(cv_pred_dir,cv_pred_inv).round(2))

r_dir-inv:  -1.0
bias:  -0.0
